### Question
The trades file contains trade economics for two options and the task is to calculate the present value for each of them using standard models. Some guidelines:

We suggest you to write a python script to solve this.

•	Price it with Mark-to-market date 2021-08-31, which basically means calculate the present value on 2021-08-31 (the market data should be assumed to be closing prices for that date).

•	The trades file contains a column “daycount_convention” which might not be familiar. Here the use case is to calculate how many years (or what fraction of a year) there is between two dates. For example, day count convention 30/360 treats a month as 30 days and a year as 360 days which means that 

o	Between 2021-05-31 and 2021-08-31 it is 3 * 30 / 360 = 0.25 years.

o	Between 2021-01-31 and 2021-03-31 it is 2 * 30 / 360 = 1/6 years.

o	Between 2021-01-31 and 2022-05-31 it is (1 * 360 + 4 * 30) / 360 = 4/3 years.


### Option
An option is a contract that gives the holder, the right but not obligation to buy or sell an underlying asset at a specified price.

$1$) Call Option: an option that gives the right, but not the obligation, to buy the underlying asset at a predetermined price.


$2)$ Put option: an option that gives the right, but not the obligation, to sell the underlying asset at a predetermined price. 

## Option Pricing Models

The mathematical models use certain variables to calculate the theoretical value of an option. The theoretical value of an option is an estimate of what an option should be worth using all known inputs.


Options may also be classified according to their exercise time:

1) European style options may be exercised only at the expiration date.


2) American style options can be exercised anytime between purchase and expiration date.

### Models
1) Black-Scholes Model

2) Binomial Option Pricing Model





### Given Data

In [1]:
import pandas as pd #library for data structures and operations
trades = pd.read_csv('trades.csv')
trades

,date,trade_id,counterparty,asset_class,instrument_type,option_type,option_style,underlying,strike,currency,exercise_date,daycount_convention
0,2021-08-31,equity_option,Bank 1,equity,option,CALL,VANILLA,ABB,330,SEK,2021-12-31,30/360
1,2021-08-31,commodity_option,Bank 2,commodity,option,PUT,VANILLA,Gold,1800,USD,2022-01-31,30/360


### Daycount Convention

o	Between 2021-08-31 and 2021-12-31 it is $\frac{4*30}{360} = \frac{1}{3}$ years.

o	Between 2021-08-31 and 2021-01-31 it is $\frac{5*30}{360} = \frac{5}{12}$ years.

In [2]:
market_data = pd.read_csv('market_data.csv')
market_data

,date,asset_class,currency,foreign_currency,name,spot value,volatilty,comment
0,2021-08-31,FX,SEK,USD,NaN,8.580,NaN,fx spot rate to convert from USD to SEK
1,2021-08-31,FX,USD,EUR,NaN,1.190,NaN,fx spot rate
2,2021-08-31,FX,EUR,GBP,NaN,1.170,NaN,fx spot rate
3,2021-08-31,IR,EUR,NaN,NaN,0.015,NaN,"constant annual interest rate, continuously co..."
4,2021-08-31,IR,SEK,NaN,NaN,0.010,NaN,"constant annual interest rate, continuously co..."
5,2021-08-31,IR,USD,NaN,NaN,0.020,NaN,"constant annual interest rate, continuously co..."
6,2021-08-31,IR,GBP,NaN,NaN,0.250,NaN,"constant annual interest rate, continuously co..."
7,2021-08-31,EQUITY,SEK,NaN,ABB,322.000,0.11,NaN
8,2021-08-31,EQUITY,USD,NaN,Apple,152.720,0.23,NaN
9,2021-08-31,EQUITY,NaN,NaN,NaN,NaN,NaN,NaN


### Pricing a plain Vanilla Call Option with Black-Scholes Model

We are pricing a plain vanilla call option, no dividends, so the Black-Scholes option pricing formula is given as:

$$C = S_0N(d_1)-Ke^{-rT}N(d_2)$$

$Ke^{-rT}$ is the present value of strike price using a continuously compounded interest rate and the formula for plain vanilla put option is:

$$P = Ke^{-rT}N(-d_2)-S_0N(-d_1)$$

where $S_0$ is the stock price, $K$ is the strike price, $r$ is the risk-free interest rate, $T$ is time to maturity, $N(\cdot)$ is a normal distribution function.

$d_1$ and $d_2$ are calculated as

$$ d_1 =\frac{log(S_0/K)+ (r+0.5\sigma^2)T}{\sigma \sqrt{T}},\qquad $$

$$ d_2 = d_1-\sigma \sqrt{T}$$

We will solve for $d_2$ by using $d_1$
$$ d_2 = d_1-\sigma \sqrt{T}$$

$$d_2 =\frac{log(S_0/K)+ (r+0.5\sigma^2)T}{\sigma \sqrt{T}} -\sigma \sqrt{T}$$

$$d_2 =\frac{log(S_0/K)+ (r+0.5\sigma^2)T -\sigma^2 T}{\sigma \sqrt{T}} $$

$$d_2 =\frac{log(S_0/K)+ rT+0.5\sigma^2T-\sigma^2 T}{\sigma \sqrt{T}} $$

$$d_2 =\frac{log(S_0/K)+ (r-0.5\sigma^2)T}{\sigma \sqrt{T}} $$

The payoff of a call option is:
$$Max(S_0-K, 0)$$

so, whenever the price of the underlying $S_0$ is above the strike price $K$, we would exercise the option because the price you can pay for the asset is lower than the current market price of the asset.
Which means if $S_0 − K$ is positive for a call option you would be able to buy the asset for $K$ and then sell it for $S_0$, thus earning $S_0 − K$ in profit.

The payoff of a put option is:
$$Max(K-S_0, 0)$$


The put options work in reverse to call options. A put option is in the money when the price of the underlying $S_0$ is less than the strike price $K$. This is because you can buy the shares on the market and sell them to the option writer, who has to pay you the higher strike price. 

If the call or put options are unprofitable, they will simply be allowed to expire without exercising, and
thus will have a price of zero 

In [3]:
import numpy as np               #library that provides mathematical operations on arrays
from math import log, sqrt, exp  # common math functions
from scipy import stats          #SciPy sub-package mainly used for probabilistic distributions and statistical operations

#### Black-Scholes Option Pricing Model pioneered by Black-Scholes-Merton (1973)
use %load BSM_script.py to load the python script

In [1]:
def BlackScholes(option,S0,K,T,r,sigma):

    ''' Valuation of European vanilla call option in BSM model.
    Parameters
    ==========
    S0    : initial stock/underlying asset price
    K     : strike price
    T     : maturity date (in year fractions)
    r     : continuously compounded risk-free rate
    sigma : volatility 
    ==========
    Returns the present value of the European call or put option'''
    from math import log, sqrt, exp
    from scipy import stats
    S0 = float(S0)
    d1 = (log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = (log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    if option =='c':
        return S0 * stats.norm.cdf(d1)- K * exp(-r * T) * stats.norm.cdf(d2)
        #stats.norm.cdf — cumulative distribution function for normal distribution
    elif option =='p':
        return  K * exp(-r * T) * stats.norm.cdf(-d2)-S0 * stats.norm.cdf(-d1)
    

In [2]:
S0=322 #current stock or underlying price
K=330 #strike price
r=0.01 #risk free rate
sigma=0.11 #volatility
T=(1/3)  #time to maturity
option = 'c'
print("Call option price using BSM is ",BlackScholes(option,S0,K,T,r,sigma))

Call option price using BSM is  5.2486765823815205


In [3]:
S0=1809.68 #current stock or underlying price
K=1800 #strike price
r=0.02 #risk free rate
sigma=0.14 #volatility
T=(5/12)  #time to maturity
option = 'p'
print("Put option price using BSM is ",BlackScholes(option,S0,K,T,r,sigma))

Put option price using BSM is  53.21048169635071


#### Binomial Option Pricing Model pioneered by Cox, Ross, and Rubinstein (1979)

Suppose an option with maturity T and strike K is to be priced, using a binomial tree with N time increments on a stock with spot price S with volatility σ when the risk free rate is r.

• The stock moves up in increments of  $u = \exp^{𝜎 \sqrt dt}$

• The stock moves down in increments of $d = \frac {1}{u} = \exp^{-𝜎 \sqrt dt}$

• Each time step of length is $dt = \frac {T}{N}$

• The probability of an up move is $p = \frac{\exp^{r × dt} - d}{(u - d)}$ 

• The probability of a down move is $1 - p$. 

European call and put options are priced at the final time step as:
$$ E = \exp^{-r ×dt} (O_{u}  × p_{u}  + 0_{d}  × p_{d})$$

where $O_{u}, O_{d}$ are the option prices at next step after up and down move, $p_{u}$ is probability up and $p_{d}$ is probabilitity down. The discount factor is used to account for time value of money.

In [4]:
##Cox_Ross_Rubinstein binomial model
def Cox_Ross_Rubinstein_Tree (S,K,T,r,sigma,N, option):
    ''' Valuation of European vanilla call option in CRR model.
    Parameters
    ==========
    S     : initial stock/underlying price
    K     : strike price
    T     : maturity date (in year fractions)
    N     : number of binomial steps
    r     : continuously compounding risk-free rate
    sigma : volatility 
    ==========
    Returns the present value of the European call or put option'''
    import numpy as np 
    from math import sqrt, exp
    
    dt = T / N                   #time step of length
    u=exp(sigma*sqrt(dt))        #up factor
    d=exp(-sigma*sqrt(dt))       #down factor
    pu=((exp(r*dt))-d)/(u-d)     #probability of a price rise
    pd=1-pu                      #probability of a price fall
    df=exp(-r*dt)                #discount rate

    St = np.zeros((N+1))         #an array of 0, size N+ 1,[0, 0, 0], to save stock prices
    C = np.zeros((N+1))          #an array of zeros to save option price
    St[0]=S*d**N                 #stock price * down factor^number of steps= 322*(0.98^10)=263.4,save at index[0] 

    #for loop to fill array St,  start from 1 and end at 2
    for j in range(1, N+1): 
        St[j] = St[j-1] * u/d    #save the values,starting from index 1 to N+1, we will have 2 values
        
    # payoff for options 
    for j in range(1, N+1):
        if option   == 'c':
            C[j] = max(St[j]-K,0) #max(S0-K,0)
        elif option == 'p':
            C[j] = max(K-St[j],0) #max(K-S0,0)
            
    #loop start from 2 and end at 1 if N= 2
    for i in range(N, 0, -1):
        #loop start from 0 and end at 1
        for j in range(0, i):
            C[j] = df*(pu*C[j+1]+pd*C[j])
                      
    return C[0]


In [5]:
S=322 #current stock or underlying price
K=330 #strike price
r=0.01 #risk free rate
sigma=0.11 #volatility
T=(1/3)  #time to maturity
N=2
option = 'c'
print("Call option price using CRR is ",Cox_Ross_Rubinstein_Tree (S,K,T,r,sigma,N, option))

Call option price using CRR is  5.710239512306439


In [6]:
S=1809.68 #current stock or underlying price
K=1800 #strike price
r=0.02 #risk free rate
sigma=0.14 #volatility
T=(5/12)  #time to maturity
option = 'p' 
N = 10
print("Put option price using CRR is ",Cox_Ross_Rubinstein_Tree (S,K,T,r,sigma,N, option))

Put option price using CRR is  52.29811264327736
